In [170]:
import numpy as np
import os
import PIL
import cv2 as cv
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import seaborn as sns
plt.rcParams['figure.figsize'] = (10,5)


"""
Challenging patients
131283,197476,217636,232742,293936,318984,428903,435714,521105,
557910,559832,587463,609078,713916,716744,756515,824987,840313,
931912,961520,983053,988464

"""


In [2]:
class PerformanceMetrics(object):
    """
    Class, which has methods for measuring all basic performance metrics for semantic segmentation:
    - Pixel accuracy;
    - Mean accuracy;
    - Mean IoU;
    - frequency_weighted_IU
    
    Takes collapsed masks as an input(after used np.argmax()), where dimention of the image should be width x heigts and
    the set of pixel values from 0 to number of classes. 
    """
    def pixel_accuracy(self, eval_segm, gt_segm):
        '''
        sum_i(n_ii) / sum_i(t_i)
        '''

        self.check_size(eval_segm, gt_segm)

        cl, n_cl = self.extract_classes(gt_segm)
        eval_mask, gt_mask = self.extract_both_masks(eval_segm, gt_segm, cl, n_cl)

        sum_n_ii = 0
        sum_t_i  = 0

        for i, c in enumerate(cl):
            curr_eval_mask = eval_mask[i, :, :]
            curr_gt_mask = gt_mask[i, :, :]

            sum_n_ii += np.sum(np.logical_and(curr_eval_mask, curr_gt_mask))
            sum_t_i  += np.sum(curr_gt_mask)

        if (sum_t_i == 0):
            pixel_accuracy_ = 0
        else:
            pixel_accuracy_ = sum_n_ii / sum_t_i

        return pixel_accuracy_

    def mean_accuracy(self, eval_segm, gt_segm):
        '''
        (1/n_cl) sum_i(n_ii/t_i)
        '''

        self.check_size(eval_segm, gt_segm)

        cl, n_cl = self.extract_classes(gt_segm)
        eval_mask, gt_mask = self.extract_both_masks(eval_segm, gt_segm, cl, n_cl)

        accuracy = list([0]) * n_cl

        for i, c in enumerate(cl):
            curr_eval_mask = eval_mask[i, :, :]
            curr_gt_mask = gt_mask[i, :, :]

            n_ii = np.sum(np.logical_and(curr_eval_mask, curr_gt_mask))
            t_i  = np.sum(curr_gt_mask)

            if (t_i != 0):
                accuracy[i] = n_ii / t_i

        mean_accuracy_ = np.mean(accuracy)
        return mean_accuracy_

    def mean_IU(self, eval_segm, gt_segm):
        '''
        (1/n_cl) * sum_i(n_ii / (t_i + sum_j(n_ji) - n_ii))
        '''

        self.check_size(eval_segm, gt_segm)

        cl, n_cl   = self.union_classes(eval_segm, gt_segm)
        _, n_cl_gt = self.extract_classes(gt_segm)
        eval_mask, gt_mask = self.extract_both_masks(eval_segm, gt_segm, cl, n_cl)

        IU = list([0]) * n_cl

        for i, c in enumerate(cl):
            curr_eval_mask = eval_mask[i, :, :]
            curr_gt_mask = gt_mask[i, :, :]

            if (np.sum(curr_eval_mask) == 0) or (np.sum(curr_gt_mask) == 0):
                continue

            n_ii = np.sum(np.logical_and(curr_eval_mask, curr_gt_mask))
            t_i  = np.sum(curr_gt_mask)
            n_ij = np.sum(curr_eval_mask)

            IU[i] = n_ii / (t_i + n_ij - n_ii)

        mean_IU_ = np.sum(IU) / n_cl_gt
        return mean_IU_

    def frequency_weighted_IU(self, eval_segm, gt_segm):
        '''
        sum_k(t_k)^(-1) * sum_i((t_i*n_ii)/(t_i + sum_j(n_ji) - n_ii))
        '''

        self.check_size(eval_segm, gt_segm)

        cl, n_cl = self.union_classes(eval_segm, gt_segm)
        eval_mask, gt_mask = self.extract_both_masks(eval_segm, gt_segm, cl, n_cl)

        frequency_weighted_IU_ = list([0]) * n_cl

        for i, c in enumerate(cl):
            curr_eval_mask = eval_mask[i, :, :]
            curr_gt_mask = gt_mask[i, :, :]

            if (np.sum(curr_eval_mask) == 0) or (np.sum(curr_gt_mask) == 0):
                continue

            n_ii = np.sum(np.logical_and(curr_eval_mask, curr_gt_mask))
            t_i  = np.sum(curr_gt_mask)
            n_ij = np.sum(curr_eval_mask)

            frequency_weighted_IU_[i] = (t_i * n_ii) / (t_i + n_ij - n_ii)

        sum_k_t_k = self.get_pixel_area(eval_segm)

        frequency_weighted_IU_ = np.sum(frequency_weighted_IU_) / sum_k_t_k
        return frequency_weighted_IU_

    def get_pixel_area(self, segm):
        return segm.shape[0] * segm.shape[1]

    def extract_both_masks(self, eval_segm, gt_segm, cl, n_cl):
        eval_mask = self.extract_masks(eval_segm, cl, n_cl)
        gt_mask   = self.extract_masks(gt_segm, cl, n_cl)

        return eval_mask, gt_mask

    def extract_classes(self,segm):
        cl = np.unique(segm)
        n_cl = len(cl)

        return cl, n_cl

    def union_classes(self, eval_segm, gt_segm):
        eval_cl, _ = self.extract_classes(eval_segm)
        gt_cl, _   = self.extract_classes(gt_segm)

        cl = np.union1d(eval_cl, gt_cl)
        n_cl = len(cl)

        return cl, n_cl

    def extract_masks(self, segm, cl, n_cl):
        h, w  = self.segm_size(segm)
        masks = np.zeros((n_cl, h, w))

        for i, c in enumerate(cl):
            masks[i, :, :] = segm == c

        return masks

    def segm_size(self, segm):
        try:
            height = segm.shape[0]
            width  = segm.shape[1]
        except IndexError:
            raise

        return height, width

    def check_size(self, eval_segm, gt_segm):
        h_e, w_e = self.segm_size(eval_segm)
        h_g, w_g = self.segm_size(gt_segm)

        if (h_e != h_g) or (w_e != w_g):
            raise EvalSegErr("DiffDim: Different dimensions of matrices!")


    class EvalSegErr(Exception):
        '''
        Class exception 
        '''
        def __init__(self, value):
            self.value = value

        def __str__(self):
            return repr(self.value)



# Base class evaluator

In [3]:
class BaseEvaluator(object):
    
    def __init__(self, prediction_dict_path):
        self.dict_of_prediction_data = pickle.load(open(prediction_dict_path, "rb"))
        self.predictions = self.dict_of_prediction_data['predictions']
#         self.ground_truth = self.dict_of_prediction_data['y']
        self.images = self.dict_of_prediction_data['x']
#         self.history_of_training = self.dict_of_prediction_data['history_of_training']
        self.number_of_test_images = len(self.predictions)
    
    def __len__(self):
        return self.number_of_test_images

    def display_training_history(self):
        """
        Draw 2 figures(validation and training info) about loss and accuracy. 
        """
        loss_hist = [s for s in self.history_of_training.keys() if 'loss' in s and 'val' not in s]
        val_loss_list = [s for s in self.history_of_training.keys() if 'loss' in s and 'val' in s]
        acc_list = [s for s in self.history_of_training.keys() if 'acc' in s and 'val' not in s]
        val_acc_list = [s for s in self.history_of_training.keys() if 'acc' in s and 'val' in s]

        if len(loss_hist) == 0:
            print('Missing history')
            return

        epochs = range(1,len(self.history_of_training[loss_hist[0]]) + 1)

        # loss
        fig = plt.figure(frameon=False)
        ax = plt.gca()
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        for l in loss_hist:
            plt.plot(epochs, 
                     self.history_of_training[l], 
                     'b', 
                     label='Training loss ('+str(str(format(self.history_of_training[l][-1],'.5f'))+')'),
                     linewidth=3
                    )
        for l in val_loss_list:
            plt.plot(epochs, 
                     self.history_of_training[l], 
                     'g',
                     label='Validation loss ('+str(str(format(self.history_of_training[l][-1],'.5f'))+')'),
                     linewidth=3
                    )
        plt.title('Loss', fontsize=50, fontdict={'fontweight':'bold'})
        plt.xlabel('Epochs',fontsize=20, fontdict={'fontweight':'bold'})
        plt.ylabel('Loss',fontsize=20, fontdict={'fontweight':'bold'})
        plt.tick_params(axis='both', which='major', labelsize=20)
        plt.grid(b=True)
        plt.legend(fontsize=30)

        # accuracy
        fig1 = plt.figure(frameon=False)
        ax1 = plt.gca()
        ax1.spines['top'].set_visible(False)
        ax1.spines['right'].set_visible(False)
        for l in acc_list:
            plt.plot(epochs, self.history_of_training[l], 'b', 
                     label='Training accuracy ('+str(format(self.history_of_training[l][-1],'.5f'))+')',
                     linewidth=3
                    )
        for l in val_acc_list:    
            plt.plot(epochs, self.history_of_training[l], 'g', 
                     label='Validation accuracy ('+str(format(self.history_of_training[l][-1],'.5f'))+')',
                     linewidth=3
                    )

        plt.title('Accuracy',fontsize=50, fontdict={'fontweight':'bold'})
        plt.xlabel('Epochs',fontsize=20, fontdict={'fontweight':'bold'})
        plt.ylabel('Accuracy',fontsize=20, fontdict={'fontweight':'bold'})
        plt.tick_params(axis='both', which='major', labelsize=20)
        plt.grid(b=True)

        plt.legend(loc=4, fontsize=30)
        plt.show()

# Sigmoid Evaluator

In [4]:
class SigmoidEvaluator(BaseEvaluator):
    
    def __init__(self, prediction_dict_path, best_threshold=0.5):
        BaseEvaluator.__init__(self,prediction_dict_path)
        self.best_threshold = best_threshold

    def threshold(self, index, threshold=0.5):
        """
        Return thresholded img where is only 0s and 1s without probabilities
        """
        thresholded_img = (self.predictions[index] > threshold).astype(np.uint8)
        thresholded_img[thresholded_img == 1] = 255
        return thresholded_img

    def display_raw_probabilities_mask(self, index):
        """
        Draw predicted probabilities of the model (without any thresholding).
        """
        plt.figure(figsize=(10,10))
        plt.axis("Off")
        plt.imshow(self.predictions[index])

    def display_original_img_vs_raw_prediction(self, index):
        """
        Draw image - prediction probability pair. 
        """
        plt.figure(figsize=(20,10))
        plt.subplot(1,2,1)
        plt.title('Image', fontweight='bold', fontsize=30)
        plt.axis("Off")
        plt.imshow(self.images[index])

        plt.subplot(1,2,2)
        plt.title("Predicted probabily mask",fontweight='bold', fontsize=30)
        plt.axis("Off")
        plt.imshow(self.predictions[index])

    def iou_for_one_image(self, index):
        intersection = np.logical_and(self.ground_truth[index], self.predictions[index])
        union = np.logical_or(self.ground_truth[index], self.predictions[index])
        iou_score = np.sum(intersection) / np.sum(union)
        return iou_score
    
    def iou_score(self):
        iou = 0
        for i in range(len(self.predictions)):
            iou += iou_for_one_image(i)
        return iou / len(self.predictions)

    
class SigmoidMulticalssModelEvaluation(SigmoidEvaluator):
    
    def __init__(self, prediction_dict_path, best_threshold=0.5):
        SigmoidEvaluator.__init__(self, prediction_dict_path, best_threshold)
        

class SigmoidOneClassModelEvaluation(SigmoidEvaluator):
    
    def __init__(self, prediction_dict_path, best_threshold=0.5):
        SigmoidEvaluator.__init__(self, prediction_dict_path, best_threshold)

# Softmax Evaluator

In [171]:
class SoftmaxEvaluator(BaseEvaluator):
    
    def __init__(self, prediction_dict_path):
        BaseEvaluator.__init__(self,prediction_dict_path)
        self.performanceMetrics = PerformanceMetrics()
#         self.doctors_predict = float(self.dict_of_prediction_data['name_of_the_dataset'].split('_')[1]) / 100
        self.doctors_predict = procento.loc[int(self.dict_of_prediction_data['name_of_the_dataset'])]['procento_stenozy'] / 100
    def get_mask(self, index):
        return np.copy(self.ground_truth[index])
    
    def get_prediction(self, index):
        return np.copy(self.predictions[index])
    
    def get_image(self, index):
        return np.copy(self.images[index])
    
    def collapse_mask(self, index, isGroundTruth=False):
        """
        Apply np.argmax function to the prediction mask to get 1-D image. 
        """
        if isGroundTruth:
            return np.argmax(self.ground_truth[index], axis=-1)
        return np.argmax(self.predictions[index], axis=-1)
        
    def prepare_mask_for_display(self, index, isGroundTruth=False):
        if isGroundTruth:
            collapsed_prediction = self.collapse_mask(index, isGroundTruth)
        else:
            collapsed_prediction = self.collapse_mask(index)
        collapsed_prediction+=1 # moving class index from 0 to 1 for making so that tisssue=1, lumen=2, artifacts=3
        collapsed_prediction[collapsed_prediction==4] = 0 # making background as a 0 class
        return collapsed_prediction

    def mean_iou_per_image(self, index):
        return self.performanceMetrics.mean_IU(self.prepare_mask_for_display(index),
                                               self.prepare_mask_for_display(index, isGroundTruth=True)
                                              )
    
    def pixel_accuracy_per_image(self, index):
        return self.performanceMetrics.pixel_accuracy(self.prepare_mask_for_display(index),
                                                      self.prepare_mask_for_display(index, isGroundTruth=True)
                                                     )
    
    def mean_accuracy_per_image(self, index):
        return self.performanceMetrics.mean_accuracy(self.prepare_mask_for_display(index),
                                                     self.prepare_mask_for_display(index, isGroundTruth=True)
                                                    )
    
    def frequency_weighted_iou_per_image(self, index):
        return self.performanceMetrics.frequency_weighted_IU(self.prepare_mask_for_display(index),
                                                             self.prepare_mask_for_display(index, isGroundTruth=True)
                                                            )
    
    def _calulate_metric_over_all_test_set(self, metric_function):
        """
        Return calculated metric of a given function over all test set.
        """
        iou = 0
        for i in range(self.number_of_test_images):
            iou+= metric_function(i)
        iou /= self.number_of_test_images
        return iou
    
    def get_mean_iou(self):
        """
        Return Mean IoU of all test set. 
        """
        return self._calulate_metric_over_all_test_set(self.mean_iou_per_image)
    
    def get_pixel_accuracy(self):
        """
        Return Pixel Accuracy of all test set. 
        """
        return self._calulate_metric_over_all_test_set(self.pixel_accuracy_per_image)
    
    def get_mean_accuracy(self):
        """
        Return Mean Accuracy of all test set.
        """
        return self._calulate_metric_over_all_test_set(self.mean_accuracy_per_image)
    
    def get_frequency_weighted_iou(self):
        """
        Return Frequency Weighted IoU of all test set.
        """
        return self._calulate_metric_over_all_test_set(self.frequency_weighted_iou_per_image)
    
    def get_evaluation(self):
        evaluation_metrics = {}
        evaluation_metrics['Frequency weighted IoU'] = self.get_frequency_weighted_iou()
        evaluation_metrics['Mean IoU'] = self.get_mean_iou()
        evaluation_metrics['Mean Accuracy'] = self.get_mean_accuracy()
        evaluation_metrics['Pixel Accuracy'] = self.get_pixel_accuracy()
        return evaluation_metrics
    
    def display_mask(self, index):
        prepared_mask = self.prepare_mask_for_display(index, isGroundTruth=True)
        plt.figure(figsize=(10,10))
        plt.imshow(prepared_mask, cmap='seismic')
        
    def display_prediction(self, index):
        prepared_prediction = self.prepare_mask_for_display(index)
        plt.figure(figsize=(10,10))
        plt.imshow(prepared_prediction, cmap='seismic')
        
    def display_prediction_mask_pair(self, index):
        prepared_prediction = self.prepare_mask_for_display(index)
        prepared_mask = self.prepare_mask_for_display(index, isGroundTruth=True)

        plt.figure(figsize=(20,10))
        plt.subplot(1,2,1)
        plt.title('Ground Truth', fontweight='bold', fontsize=30)
        plt.imshow(prepared_mask, cmap='seismic')
        plt.subplot(1,2,2)
        plt.title('Prediction', fontweight='bold', fontsize=30)
        plt.imshow(prepared_prediction, cmap='seismic')
    
    def display_original_img_prediction_pair(self, index):
        prepared_prediction = self.prepare_mask_for_display(index)
        
        plt.figure(figsize=(20,10))
        plt.subplot(1,2,1)
        plt.title('Original Image', fontweight='bold', fontsize=30)
        plt.imshow(self.images[index])
        plt.subplot(1,2,2)
        plt.title('Prediction', fontweight='bold', fontsize=30)
        plt.imshow(prepared_prediction, cmap='seismic')
    
    def display_mask_prediction(self, index):
        prepared_prediction = self.prepare_mask_for_display(index)
        lumen = np.copy(prepared_prediction)
        lumen[lumen!=2] = 0
        lumen[lumen==2] = 255

        artifacts = np.copy(prepared_prediction)
        artifacts[artifacts!=3] = 0
        artifacts[artifacts==3] = 255

        tissue = np.copy(prepared_prediction)
        tissue[tissue!=1] = 0
        tissue[tissue==1] = 255
        
        prepared_mask = self.prepare_mask_for_display(index, isGroundTruth=True)
        lumen_mask = np.copy(prepared_mask)
        lumen_mask[lumen_mask!=2] = 0
        lumen_mask[lumen_mask==2] = 255

        artifacts_mask = np.copy(prepared_mask)
        artifacts_mask[artifacts_mask!=3] = 0
        artifacts_mask[artifacts_mask==3] = 255

        tissue_mask = np.copy(prepared_mask)
        tissue_mask[tissue_mask!=1] = 0
        tissue_mask[tissue_mask==1] = 255
        
        final_mask = np.dstack((tissue, lumen, artifacts)).astype(np.uint8)
        
        _, tissue_countours, _ = cv.findContours(tissue_mask.astype(np.uint8), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        _, artifact_countours, _ = cv.findContours(artifacts_mask.astype(np.uint8), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        _, lumen_countours, _ = cv.findContours(lumen_mask.astype(np.uint8), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
                
        cv.drawContours(final_mask, tissue_countours, -1, (255,180,180), 2)
        cv.drawContours(final_mask, lumen_countours, -1, (180,255,180), 2)
        cv.drawContours(final_mask, artifact_countours, -1, (180,180,255), 2)
        return final_mask
    
    def display_merged_img_prediction(self, index, intensity=0.1):
        prepared_prediction = self.prepare_mask_for_display(index)
        lumen = np.copy(prepared_prediction)
        lumen[lumen!=2] = 0
        lumen[lumen==2] = 255

        artifacts = np.copy(prepared_prediction)
        artifacts[artifacts!=3] = 0
        artifacts[artifacts==3] = 255

        tissue = np.copy(prepared_prediction)
        tissue[tissue!=1] = 0
        tissue[tissue==1] = 255

        final_mask = np.dstack((tissue, lumen, artifacts)).astype(np.uint8)
        _, tissue_countours, _ = cv.findContours(tissue.astype(np.uint8), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        _, artifact_countours, _ = cv.findContours(artifacts.astype(np.uint8), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        _, lumen_countours, _ = cv.findContours(lumen.astype(np.uint8), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

        final_mask = cv.addWeighted(np.copy(self.images[index]),1,final_mask,intensity, 0)
        cv.drawContours(final_mask, tissue_countours, -1, (255,0,0), 1)
        cv.drawContours(final_mask, lumen_countours, -1, (0,255,0), 1)
        cv.drawContours(final_mask, artifact_countours, -1, (0,0,255), 1)
#                 plt.figure(figsize=(10,10))
#         plt.imshow(cv.addWeighted(a.images[index],1,final_mask,intensity, 0))
        return final_mask

    def _helper_process_patient(self, index, intensity=0.1):
        prepared_prediction = self.prepare_mask_for_display(index)
        lumen = np.copy(prepared_prediction)
        lumen[lumen!=2] = 0
        lumen[lumen==2] = 255

        artifacts = np.copy(prepared_prediction)
        artifacts[artifacts!=3] = 0
        artifacts[artifacts==3] = 255

        tissue = np.copy(prepared_prediction)
        tissue[tissue!=1] = 0
        tissue[tissue==1] = 255

        final_mask = np.dstack((tissue, lumen, artifacts)).astype(np.uint8)
        _, tissue_countours, _ = cv.findContours(tissue.astype(np.uint8), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        _, artifact_countours, _ = cv.findContours(artifacts.astype(np.uint8), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        _, lumen_countours, _ = cv.findContours(lumen.astype(np.uint8), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

        final_mask = cv.addWeighted(np.copy(self.images[index]),1,final_mask,intensity, 0)
        cv.drawContours(final_mask, tissue_countours, -1, (255,0,0), 1)
        cv.drawContours(final_mask, lumen_countours, -1, (0,255,0), 1)
        cv.drawContours(final_mask, artifact_countours, -1, (0,0,255), 1)
        font = cv.FONT_HERSHEY_SIMPLEX
        if (np.count_nonzero(tissue) + np.count_nonzero(lumen)) == 0:
            percentage_stenosis = 1.0
            cv.putText(final_mask,"Hard to segment",(10,50),font, 1,(255,255,255),2,cv.LINE_AA)
        else:
            percentage_stenosis = (np.count_nonzero(tissue) / (np.count_nonzero(tissue) + np.count_nonzero(lumen)))
            if percentage_stenosis >=0.99:
                cv.putText(final_mask,"Hard to segment",(10,50),font, 1,(255,255,255),2,cv.LINE_AA)
            else:
                cv.putText(final_mask,"Percentage of stenosis: "+str(np.round(percentage_stenosis,2)),(10,50), 
                           font, 1,(255,255,255),2,cv.LINE_AA)
                cv.putText(final_mask,"Your estimation: "+str(self.doctors_predict),(10,100), 
                           font, 1,(50,255,50),2,cv.LINE_AA)
        orig_img = my_eval.get_image(index)
        cv.putText(orig_img,"Original Image",(10,50), font, 1,(255,255,255),2,cv.LINE_AA)
        return (percentage_stenosis,final_mask, orig_img)
    
    def process_patient(self):
        results = []
        for i in range(self.number_of_test_images):
            results.append(self._helper_process_patient(i))
        
        percentages_stenosis = [x[0] for x in results if x[0] < 0.99]
        hard_to_segment = [x[1:] for x in results if x[0] >0.99]
        norm_to_segmnet = [x[1:] for x in results if x[0]<=0.99]
        
        return (percentages_stenosis, hard_to_segment, norm_to_segmnet)

def concat_images(imga, imgb):
    """
    Combines two color image ndarrays side-by-side.
    """
    ha,wa = imga.shape[:2]
    hb,wb = imgb.shape[:2]
    max_height = np.max([ha, hb])
    total_width = wa+wb
    new_img = np.zeros(shape=(max_height, total_width, 3))
    new_img[:ha,:wa]=imga
    new_img[:hb,wa:wa+wb]=imgb
    return new_img.astype(np.uint8)

# Estimating

In [172]:
PATH_TO_SAVE = '/home/bohdan/ultrasound/patients/'
PREDICTION_DICT_PATH = '/home/bohdan/ultrasound/predictions/patients/'
datasets_names = os.listdir(PREDICTION_DICT_PATH)
datasets_names = [x.split('.')[0] for x in datasets_names]

In [195]:
PATH_TO_TABLE = '/home/bohdan/Desktop/platy.xlsx'
platy = pd.read_excel(PATH_TO_TABLE)
procento = platy[['id','procento_stenozy']]
procento = procento[np.logical_not(pd.isnull(procento['procento_stenozy']))]

for i in procento.index:
    if not isinstance(procento.loc[i]['procento_stenozy'], int):
        procento.drop(i, inplace=True)

procento.set_index(procento['id'], inplace=True)
procento.drop('id', axis=1, inplace=True)
procento.drop([452770,921310], inplace=True)# don`t have the ultrasound data
procento.drop([220574,890878], inplace=True) # bad quality

In [196]:
def plot_patient_stenosis_perscntage(my_eval, percentages_stenosis, path_to_save):
    f = plt.figure(figsize=(25,10))
    ax = f.add_subplot(111)

    plt.xlabel('Images', fontweight='bold',fontsize=20)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    sns.set(style="whitegrid")
    
    plt.title("Patient ID: {}".format(my_eval.dict_of_prediction_data['name_of_the_dataset']),
              fontweight='bold',fontsize=30)
    
    xt = ax.get_yticks() 
    xt=np.append(xt, my_eval.doctors_predict)
    xtl=xt.tolist()
    xtl = [np.round(x,1) for x in xtl]
    xtl[-1]="Your Percentage of stenosis: {}".format(str(my_eval.doctors_predict))
    ax.set_yticks(xt)
    ax.set_yticklabels(xtl)

    plt.ylim((0,1.1))
    plt.plot(percentages_stenosis,marker='o', color='blue', linewidth=5, alpha=0.7, markersize=15)
    plt.axhline(y=my_eval.doctors_predict, ls="--", c="red", linewidth=3)
    plt.savefig(path_to_save)
    plt.close('all')

In [177]:
hard_to_interpret = {}
persentages_of_stenosys_of_all_patients = {}
for i in tqdm_notebook(range(len(datasets_names))):
    my_eval = SoftmaxEvaluator(
        PREDICTION_DICT_PATH+datasets_names[i]+'.dat'
    )
    percentages_stenosis, hard_to_segment, norm_to_segment = my_eval.process_patient()
    hard_to_interpret[my_eval.dict_of_prediction_data['name_of_the_dataset']] = hard_to_segment
    
    path_for_new_patient = os.path.join(PATH_TO_SAVE,datasets_names[i])
    if not os.path.exists(path_for_new_patient):
        os.mkdir(path_for_new_patient)
    
    segmentation_path = os.path.join(path_for_new_patient, 'segmentation_results')
    if not os.path.exists(segmentation_path):
        os.mkdir(segmentation_path)
    
    path_to_save_figure=path_for_new_patient+'/'+datasets_names[i]+'.png'
    plot_patient_stenosis_perscntage(my_eval, percentages_stenosis, path_to_save_figure)
    
    persentages_of_stenosys_of_all_patients[my_eval.dict_of_prediction_data['name_of_the_dataset']] = percentages_stenosis
    
    for j in range(len(norm_to_segment)):
        norm_to_segment_result = norm_to_segment[j][0]
        norm_to_segment_original = norm_to_segment[j][1]
        merged_img = concat_images(norm_to_segment_original[..., ::-1],norm_to_segment_result[..., ::-1])
        cv.imwrite(segmentation_path+'/'+str(j)+'.png',merged_img)
#     break

HBox(children=(IntProgress(value=0, max=111), HTML(value='')))

In [197]:
whole_error = 0
for i in range(len(procento)):
    right_prediction = (procento.loc[int(datasets_names[i])] / 100)['procento_stenozy']
    our_prediction = persentages_of_stenosys_of_all_patients[datasets_names[i]]
    our_prediction.sort()
    
    if len(our_prediction) == 0:
#             print(datasets_names[i])
            continue
    elif len(our_prediction) <= 3:
        our_prediction = np.mean(our_prediction)
    else:
        our_prediction = np.mean(our_prediction[:4])
    whole_error+= abs(our_prediction - right_prediction)
#     print(datasets_names[i],": ",right_prediction, ' ', our_prediction, len(persentages_of_stenosys_of_all_patients[datasets_names[i]]))
print(whole_error)

17.439723887675903


# Save the results to table

In [199]:
your_percentage_of_stenosis = []
our_percentage_of_stenosis = []
indexes_to_remove = []
for i in range(len(procento)):
    right_prediction = (procento.loc[int(datasets_names[i])] / 100)['procento_stenozy']
    our_prediction = persentages_of_stenosys_of_all_patients[datasets_names[i]]
    our_prediction.sort()
    
    if len(our_prediction) == 0:
            indexes_to_remove.append(datasets_names[i])
            continue
    elif len(our_prediction) <= 3:
        our_prediction = np.mean(our_prediction)
    else:
        our_prediction = np.mean(our_prediction[:4])
    our_percentage_of_stenosis.append(our_prediction)
    your_percentage_of_stenosis.append(right_prediction)
#     print(datasets_names[i],": ",right_prediction, ' ', our_prediction, len(persentages_of_stenosys_of_all_patients[datasets_names[i]]))
indexes_to_remove = [int(x) for x in indexes_to_remove]
our_percentage_of_stenosis = [np.round(x, 2)*100 for x in our_percentage_of_stenosis]
your_percentage_of_stenosis = [int(x*100) for x in your_percentage_of_stenosis]

In [200]:
indexes = procento.index
indexes = [x for x in indexes if x not in indexes_to_remove]
d = {
    'Our Percentage of Stenosis': our_percentage_of_stenosis,
    'Your Percentage of Stenosis': your_percentage_of_stenosis,
    'Our Comments': [None for x in range(len(indexes)) ]
}
df = pd.DataFrame(
    index= indexes,
    data=d
)
df.index.name = "PatientID"
columnsTitles = ['Your Percentage of Stenosis', 'Our Percentage of Stenosis','Our Comments']
df = df.reindex(columns=columnsTitles)
df.head()

,Your Percentage of Stenosis,Our Percentage of Stenosis,Our Comments
PatientID,,,
105520,65,65.0,None
128207,30,90.0,None
131283,70,88.0,None
137930,50,63.0,None
144329,75,83.0,None


In [202]:
df.to_csv('procento_stenozy.csv')

In [ ]:
if not os.path.exists(os.path.join(PATH_TO_SAVE, 'hard_to_segment')):
    os.mkdir(os.path.join(PATH_TO_SAVE, 'hard_to_segment'))
    
for patient in hard_to_interpret:
    if not os.path.exists(os.path.join(PATH_TO_SAVE,'hard_to_segment', patient)):
        os.mkdir(os.path.join(PATH_TO_SAVE, 'hard_to_segment', patient))
    
    for i in range(len(hard_to_interpret[patient])):
        merged_img = concat_images(hard_to_interpret[patient][i][1],hard_to_interpret[patient][i][0])[..., ::-1]
        cv.imwrite(os.path.join(PATH_TO_SAVE, 'hard_to_segment', patient)+'/'+str(i)+'.png',merged_img)

### Save the results of concatenating images

In [ ]:
def save_concatenated_imgs(evaluator, path_to_save):
    if not os.path.exists(path_to_save):
        os.mkdir(path_to_save)
    images_to_stack = []
    for i in range(len(evaluator)):
        prediction_mask = evaluator.display_mask_prediction(i)
        original_prediction = evaluator.display_merged_img_prediction(i)
        images_to_stack.append((original_prediction, prediction_mask))

    for i in range(len(evaluator)):
        im = concat_images(images_to_stack[i][0],images_to_stack[i][1])
        cv.imwrite(os.path.join(path_to_save,'{}.png'.format(str(i))), im[...,::-1])

In [ ]:
for dataset_name in datasets_names:
    my_eval = SoftmaxEvaluator(
        PREDICTION_DICT_PATH+dataset_name
    )
    save_concatenated_imgs(my_eval,'/home/bohdan/ultrasound/predictions/model_estimation/{}/predictions/'
                           .format(dataset_name.split('.')[0]))
    

In [ ]:
import seaborn as sns
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size
import plotly
plotly.tools.set_credentials_file(username='BohdanPetryshak', api_key='h8ECd47FhDo1gfrRULQg')


In [ ]:
pixel_acc = []
mean_acc = []
mean_iou = []
for dataset_name in datasets_names:
    my_eval = SoftmaxEvaluator(
        PREDICTION_DICT_PATH+dataset_name
    )
    pixel_acc.append(my_eval.get_pixel_accuracy())
    mean_acc.append(my_eval.get_mean_accuracy())
    mean_iou.append(my_eval.get_mean_iou())

In [ ]:
pixel_acc

In [ ]:
labels = ['rgb_color_aug.dat',
 'rgb_non_rigid_aug.dat',
 'rgb_color_non_destructive_aug.dat',
 'rgb_all_types_of_aug.dat',
 'rgb_non_destructive_aug.dat']
lab = [0,0,0,0,0,1,1,1,1,1,2,2,2,2,2]
metrics_names = ['Pixel Accuracy', 'Mean Accuracy', 'Mean IoU']
data = pd.DataFrame({'acc': pixel_acc+mean_acc+mean_iou,
                     'names': labels+labels+labels,
                     'lab': [metrics_names[x] for x in lab]
})

In [ ]:
plt.figure(figsize=(20,10))
ax = sns.barplot(x="names", y="acc", data=data, hue='lab', palette=['#96bf80', '#6c8ebf', '#b85450'])

plt.tick_params(axis='both', which='major', labelsize=15)
ax.plot(secondary_y=['B'],grid=False)
plt.xticks(rotation=10)# fontweight='bold')
plt.yticks(fontweight='bold')
plt.xticks(fontweight='bold')
sns.set(style="whitegrid",context='poster')
plt.xlabel('Datasets \n ', fontsize=30, fontweight='bold')
plt.ylabel('IoU', fontsize=30, fontweight='bold')
# ax.spines['left'].set_visible(False)
# ax.spines['right'].set_visible(False)


In [ ]:
a = my_eval.prepare_mask_for_display(3)

In [ ]:
lumen = np.copy(a)
lumen[lumen!=2] = 0
lumen[lumen==2] = 255

artifacts = np.copy(a)
artifacts[artifacts!=3] = 0
artifacts[artifacts==3] = 255

tissue = np.copy(a)
tissue[tissue!=1] = 0
tissue[tissue==1] = 255

In [ ]:
labels = ['Pixel Accuracy', 'Mean Accuracy', 'Mean IoU']
lab = [0,1,2,3,4,5,0,1,2,3,4,5,0,1,2,3,4,5]
metrics_names = ['Color', 'Non-Rigid', 
                 'Color + Non-Destructive', 
                 'Color + Non-Rigid + Non-Destructive',
                 'Non-Destructive', 'Without augmentation']
data = pd.DataFrame({'acc': pixel_acc+mean_acc+mean_iou,
                     'names': labels[:1]*6+labels[1:2]*6+labels[2:]*6,
                     'lab': [metrics_names[x] for x in lab]
})


In [ ]:
plt.figure(figsize=(20,10))
ax = sns.barplot(x="names", y="acc", data=data, hue='lab')#, palette=['#96bf80', '#6c8ebf', '#b85450'])
plt.tick_params(axis='both', which='major', labelsize=15)
plt.ylim((0.7,1))
# ax.plot(secondary_y=['B'],grid=False)
# plt.xticks(rotation=10)# fontweight='bold')
plt.yticks(fontweight='bold')
plt.xticks(fontweight='bold')
sns.set(style="whitegrid")
plt.xlabel('Performance Metrics \n ', fontsize=30, fontweight='bold')
plt.ylabel('IoU', fontsize=30, fontweight='bold')
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.legend(fontsize=15)

In [ ]:
a = [(1,100), (2,90), (3,80)]

In [ ]:
a.sort(key=lambda x: x[1])

In [ ]:
def sinplot(flip=1):
    x = np.linspace(0, 14, 100)
    for i in range(1, 7):
        plt.plot(x, np.sin(x + i * .5) * (7 - i) * flip)

In [ ]:
x = np.linspace(0, 2*np.pi, 100)
y = np.sin(x)

ax = plt.subplot(111)
ax.plot(x, y)

# Hide the right and top spines
